In [ ]:
### Manually consuming an iterator

In [ ]:
lst = iter(list(range(10)))

try:
    while True:
        print(next(lst))
except StopIteration as si:
    pass

In [ ]:
# or without try

lst = iter(range(10))

while True:
    line = next(lst, None)  # we are telling next to return None when its done iterating 
    if line is None:
        break
    print(line)

### Using a file iterator

In [ ]:
with open("/etc/passwd", "r") as f:
    # note: f is a generator
    try:
        while True:
            entry = next(f)
            print(entry)
    except StopIteration as si:
        pass

In [2]:
# instructing next to do nothing, passing None when there is nothing more to iterate on
lst = iter(list(range(10)))
while True:
    entry = next(lst, None) # passing None prevents from throwing StopIteration exception    
    if entry is None:
        break
    print(entry, end = ' ')
    

0 1 2 3 4 5 6 7 8 9 

### Creating a custom iterator for your own container (defined using a class)

In [4]:
from __future__ import annotations 

"""
Note: before py 3.7 one would have to use 'Node' instead to delay evaluation of the type
But: Post 3.7, we can import - from __futures__ import annotations , to prepone evaluation of annotation
"""

class Node:
    def __init__(self, value: int):
        self._value : int = value
        self._children : list[Node] = []
    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    def add_children(self, node: Node):
        self._children.append(node)
    
    def __iter__(self):
        return iter(self._children)

In [5]:
node = Node(0)
child1 = Node(1)
child2 = Node(2)

node.add_children(child1)
node.add_children(child2)

for ch in node: # note: think of this as iter() being called on the node object which internally calls on nodes children
    print(ch) # this should call __repr__


Node(1)
Node(2)


In [23]:
# writing custom incrementor - floating point numbers
def frange(start, stop, increment):
    x = start
    while x < stop:
        yield x
        x += increment

In [25]:
list(frange(0.1, 5, 0.4))

[0.1,
 0.5,
 0.9,
 1.3,
 1.7000000000000002,
 2.1,
 2.5,
 2.9,
 3.3,
 3.6999999999999997,
 4.1,
 4.5,
 4.9]

In [36]:
## Implementing Iterator Protocol

class Node:
    def __init__(self, value: int):
        self._value : int = value
        self._children : list[Node] = []
    
    def __repr__(self) -> str:
        return 'Node({!r})'.format(self._value)

    def add_children(self, node: Node) -> None:
        self._children.append(node)
    
    def __iter__(self) -> Node:
        return iter(self._children)

    def depth_first(self):
        yield self # yield itself
        for ch in self:
            yield from ch.depth_first() # `yield from` is to yield values from the generator vs yield for value itself

In [37]:
root = Node(0)
child1 = Node(1)
child2 = Node(2)

root.add_children(child1)
root.add_children(child2)

child1.add_children(Node(3))
child1.add_children(Node(4))
child2.add_children(Node(5))

In [38]:
for ch in root.depth_first():
    print(ch)

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In [52]:
### Iterator a generator in reverse (custom type)

class Countdown:
    def __init__(self, start):
        self._start = start
    
    def __iter__(self):
        n = self._start
        while n > 0:
            yield n
            n -= 1

    # reverse iterator
    def __reversed__(self):
       n = 1
       while n <= self._start:
           yield n
           n += 1 


In [53]:
c = Countdown(20)

In [54]:
list(c)

[20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]

In [55]:
list(reversed(c))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

### Defining generators with extra state

In [56]:
# this is a line history example
from collections import deque

class linehistory:
    def __init__(self, lines, history_len=3):
        self.lines = lines
        self.history = deque(maxlen=history_len)
    
    def __iter__(self):
        for lineno, line in enumerate(self.lines):
            self.history.append((lineno, line))
            yield line

    def clear(self):
        self.history.clear()